**--- Part One ---**

In [98]:
## Select input data case
## Note: this assumes a plain textfile named `input_{case}` is located in the same folder as this notebook
case = "example" # <- input_example
case = "jfm"
case = "gatton"

# verbose output
verbose = False

import numpy as np
from itertools import cycle

# Read input
with open(f'input_{case}','r') as f:
    input_lines = f.readlines()

import numpy as np

dir_template = {0:'n',1:'s',2:'w',3:'e'}

class Elf():
    def __init__(self,pos,grid):
        self.pos = np.array(pos)
        self.reset_dirs()
    
    def next_dir(self):
        self.dir = next(self.dirs)

    def reset_dirs(self,dir='n'):
        self.dirs = cycle(dir_template)
        self.dir = next(self.dirs)
    
    def has_neighbor(self, grid):
        i,j = self.pos
        if sum(sum(grid[i-1:i+2,j-1:j+2]))-1:
            return True
        else:
            return False

    def propose_move(self, grid, dir=0):
        if self.has_neighbor(grid):
            i,j = self.pos
            for idx in range(4):
                if dir == 0: # North
                    if not any(grid[i-1, j-1:j+2]):
                        self.next_pos = self.pos + [-1,0]
                        return self.next_pos
                elif dir == 1: # South
                    if not any(grid[i+1, j-1:j+2]):
                        self.next_pos = self.pos + [1,0]               
                        return self.next_pos
                elif dir == 2: # West
                    if not any(grid[i-1:i+2, j-1]):
                        self.next_pos = self.pos + [0,-1]
                        return self.next_pos
                else: # East
                    if not any(grid[i-1:i+2, j+1]):
                        self.next_pos = self.pos + [0,1]
                        return self.next_pos
                dir = (dir+1) % 4
        self.next_pos = self.pos
        return self.next_pos


class Grid():
    def __init__(self):
        conv_to_num = {'.':0, '#':1}
        self.conv_to_sym = {0:'.', 1:'#'}
        self.dir_iter = cycle([i for i in range(4)])
        self.changed = True
        self.round = 0
        self.grid = np.array([[int(conv_to_num[i]) for i in row.strip()] for row in input_lines], dtype=int)
        self.elves = []
        for i,row in enumerate(self.grid):
            for j,val in enumerate(row):
                if val:
                    self.elves.append(Elf([i,j], self.grid))
        self.n_elves = len(self.elves)
        self.grow()

    def __getitem__(self,idx):
        return self.grid[idx]

    @property
    def height(self):
        return self.grid.shape[0]

    @property
    def width(self):
        return self.grid.shape[1]

    def execute_one_round(self,n = None, show=False):
        self.round += 1
        self.grow()
        self.propose_all()
        self.move_all()
    
    def execute_continuous(self):
        while self.changed:
            self.execute_one_round()

    def move_all_elves(self, delta):
        for e in self.elves:
            e.pos = e.pos + delta
    
    def propose_all(self):
        self.proposed = np.zeros(self.grid.shape)
        self.changed = False
        dir = next(self.dir_iter)
        for e in self.elves:
            old_pos = e.pos[:]
            new_pos = e.propose_move(self.grid,dir=dir)
            if not self.changed:
                if (old_pos[0] != new_pos[0]) or (old_pos[1] != new_pos[1]):
                    self.changed = True
            self.proposed[tuple(new_pos)] += 1
    
    def move_all(self):
        for e in self.elves:
            if self.proposed[tuple(e.next_pos)] == 1:
                self.grid[tuple(e.pos)] = 0
                self.grid[tuple(e.next_pos)] = 1
                e.pos = e.next_pos
    
    def trim(self):
        idx0 = []
        idx1 = []
        for e in self.elves:
            idx0.append(e.pos[0])
            idx1.append(e.pos[1])
        self.trimmed = self.grid[min(idx0):max(idx0)+1,min(idx1):max(idx1)+1]
        return self.trimmed

    def total(self):
        self.trim()
        nx,ny = self.trimmed.shape
        return nx*ny-self.n_elves

    def grow(self):
        if any(self.grid[0,:]): # top
            self.grid = np.vstack([np.zeros((1, self.width)), self.grid])
            self.move_all_elves([1,0])
        if any(self.grid[-1,:]): # bottom
            self.grid = np.vstack([self.grid, np.zeros((1, self.width))])
        if any(self.grid[:,0]): # left
            self.grid = np.hstack([np.zeros((self.height, 1)), self.grid])
            self.move_all_elves([0,1])
        if any(self.grid[:,-1]): # bottom
            self.grid = np.hstack([self.grid, np.zeros((self.height, 1))])

    def show(self,grid=None):
        grid = grid or self.grid
        for row in self.grid:
            print(''.join([self.conv_to_sym[i] for i in row]))
            

grid = Grid()

In [99]:
grid = Grid()
for i in range(10):
    grid.execute_one_round()
print(f'Total after 10 rounds: {grid.total()}')

Total after 10 rounds: 3906


**--- Part Two ---**

In [100]:
grid = Grid()
grid.execute_continuous()
print(f'Finished moving after round: {grid.round}')

Finished moving after round: 895
